## Topographische Modelle für UIW (120.109)

### Protokoll 2

### Präambel

Fügen Sie Ihren Code bzw. ihre Antworten an den Stellen ein, die mit `YOUR CODE HERE` oder \"YOUR ANSWER HERE\" gekennzeichnet sind. **Wichtig**: Bevor Sie die Aufgabe abgeben, stellen Sie sicher, dass die `raise NotImplementedError()` Zeilen aus Ihrem Code entfernt wurden, und dass Ihr Code fehlerlos durchläuft. Starten Sie dazu den Kernel neu und führen Sie alle Zellen aus (Doppelpfeilsymbol in der Menüleiste).

### Aufgabe 2a
Gegeben ist die 3D Punktwolke (*pcl.laz*) einer Airborne Laserscanning Befliegung eines kleinen Auwaldgebietes zwischen der Bundesstraße 1 und dem an der Pielach bei Loosdorf/NÖ. Die Erfassung erfolgte im laubfreien Zustand. Der Datensatz weist eine sehr gute Vegetationsdurchdringung aus, sodass Punkte sowohl auf den Baumkronen als auch am Boden vorhanden sind. 

![](3d-pcl.png)

Ziel dieser Aufgabe ist die Klassifizierung der gegebenen Punktwolke in Boden- und Nicht-Bodenpunkte, um anschließend mit den zum Boden gehörigen Punkten ein Digitales Geländemodell (DGM) zu berechnen. Lesen Sie zunächst die Daten mittels [Import](https://opals.geo.tuwien.ac.at/html/stable/ModuleImport.html) in den [OPALS Datenmanager](https://opals.geo.tuwien.ac.at/html/stable/ref_odm.html) ein und speichern Sie das Ergebnis unter dem Namen *pcl.odm*.

Analysieren Sie vorab für die Wahl einer geeigneten DGM-Gitterweite die Punktdichte ($Pkt/m^{2}$) mit den Modulen [Cell](https://opals.geo.tuwien.ac.at/html/stable/ModuleCell.html) und
[Histo](https://opals.geo.tuwien.ac.at/html/stable/ModuleHisto.html). Verwenden Sie zur Punktdichteanalyse nur die jeweils letzten Echos jedes Laserpulses.
Speichern Sie die mittlere Punktdichte in der Variablen `mittlere_punktdichte` und den Median der Punktdichte als `median_punkdichte`.  Das Modul Histo bennötigt bis auf inFile und outFile keine weiteren Parameter.
Die Parametereinstellungen für das Modul Cell können direkt aus dieser Angabe übernommen werden:
- cellSize = 2
- feature = 'pdens'
- filter = 'echo[last]'

##### Hinweis: Das Histogramm kann die durchschnittliche Anzahl an Punkten pro $m^{2}$ liefern. Für die Erstellung des digitalen Geländemodells mittels Interpolation sollten Sie eine Gitterweite wählen, die für jede Rasterzelle mehr als einen Punkt garantiert und gleichzeitig möglichst klein ist um ein glattes Ergebnis zu erzeugen.

Verwenden Sie zur Filterung der Bodenpunkte das OPALS Modul [TerrainFilter](https://opals.geo.tuwien.ac.at/html/nightly/ModuleTerrainFilter.html). Experimentieren Sie mit folgenden Parametern des Moduls, bis Sie ein zufriedenstellendes Ergebnis für das DGM erzielt haben:
- gridSize
- pyramidLevels
- filterThresholds

##### Hinweis: Beurteilen Sie das Ergebnis durch eine visuelle Analyse der DGM-Schummerung (siehe unten!)

Anschließend kann mit Hilfe des Moduls [Grid](https://opals.geo.tuwien.ac.at/html/stable/ModuleGrid.html) ein DGM erstellt werden. Verwenden Sie dafür nur die als Bodenpunkte klassifizierten Punkte. Folgende Parameter sind für eine erfolgreiche Interpolation der Daten einzustellen, wobei Sie die ersten drei Einstellungen direkt aus dieser Angabe übernehmen können, passende Werte für die anderen Parameter sind selbstständig zu finden:
- filter = 'class[ground]'
- selMode = opals.Types.SelectionMode.quadrant
- interpolation = opals.Types.GridInterpolator.movingPlanes
- gridSize
- neighbours
- searchRadius

##### Hinweis: Wenn sie bestimmte Parameter oder Funktionen nachschlagen wollen, machen Sie sich mit der [OPALS](https://opals.geo.tuwien.ac.at/html/stable/index.html) Dokumentation vertraut.

Erstellen Sie abschließend für das DGM mit Hilfe der OPALS Module [ZColor](https://opals.geo.tuwien.ac.at/html/stable/ModuleZColor.html) und [Shade](https://opals.geo.tuwien.ac.at/html/stable/ModuleShade.html), sowohl ein farbkodiertes TIF als auch ein ein Hillshade und speichern Sie die resultierenden Ergebnisse unter folgenden Dateinamen ab:
- Geländemodell: *dgm.tif*
- Schummerung: *dgm_hillshade.tif*
- Höhenkodierung: *dgm_zcolor.tif*

![](DGM_bsp.jpeg)

In [15]:
#
# Import Statements
#
import rasterio
import shapely
import fiona
import os

In [16]:
# YOUR CODE HERE
import opals
from opals import Import, TerrainFilter, Cell, Grid, Histo, Shade, ZColor, DSM, Algebra

Import.Import(inFile = 'pcl.laz', outFile = 'pcl.odm').run()

Cell.Cell(inFile = 'pcl.odm', cellSize = 2, feature = 'pdens', filter = 'echo[last]', outFile = 'punktdichte.tif').run()
his = Histo.Histo(inFile = 'punktdichte.tif')
his.run()

mittlere_punktdichte = his.histogram[0].getMean()
median_punktdichte = his.histogram[0].getMedian()

tf = TerrainFilter.TerrainFilter(inFile = 'pcl.odm')
tf.filter = 'echo[last]'
tf.robustInterpolation.gridSize = 0.6   #Löcher
tf.robustInterpolation.pyramidLevels = 4    
tf.robustInterpolation.filterThresholds = [0.1, 0.5, 1, 2.5]
tf.run()

grd = Grid.Grid(inFile = 'pcl.odm', outFile = 'dgm.tif')
grd.filter = 'class[ground]'
grd.gridSize = 0.25 #so dass ein paar Punkte in jedem Grid sind
grd.neighbours = 8 #gerade Anzahl
grd.searchRadius = 5
grd.selMode = opals.Types.SelectionMode.quadrant
grd.interpolation = opals.Types.GridInterpolator.movingPlanes
grd.run()

Shade.Shade(inFile = 'dgm.tif', outFile = 'dgm_hillshade.tif', sunPosition = [350,50], shading = opals.Types.ShadingMethod.tanaka).run()

ZColor.ZColor(inFile = 'dgm.tif', outFile = 'dgm_zcolor.tif', palFile = 'standardPal.xml', interval = 0.75).run()

[Parameter(path=inFile, values=dgm.tif, type=Path, source=interFace),
 Parameter(path=outFile, values=dgm_zcolor.tif, type=Path, source=interFace),
 Parameter(path=oFormat, values=GTiff, type=String, source=estimated),
 Parameter(path=pixelSize, values=0.25, type=float, source=estimated),
 Parameter(path=interval, values=0.75, type=Vector<float>, source=interFace),
 Parameter(path=nClasses, values=, type=uint32, source=unknown),
 Parameter(path=palFile, values=standardPal.xml, type=Path, source=interFace),
 Parameter(path=legend, values=file=dgm_zcolorLegend.svg, type=ZColorLegend, source=estimated),
 Parameter(path=scalePal, values=, type=ZColorScalePal, source=unknown),
 Parameter(path=offsetPal, values=0, type=float, source=estimated),
 Parameter(path=zRange, values=, type=Array<float,2>, source=unknown),
 Parameter(path=limit, values=, type=GridLimit, source=unknown),
 Parameter(path=resampling, values=, type=ResamplingMethod, source=unknown),
 Parameter(path=band, values=0, type=S

In [17]:
# Sichtbare Überprüfungen
assert os.path.exists('pcl.odm')
#
assert 'mittlere_punktdichte' in globals()
assert 'median_punktdichte' in globals()
#
assert 'Import' in globals()
assert 'Cell' in globals()
assert 'Histo' in globals()
assert 'TerrainFilter' in globals()
assert 'Grid' in globals()
assert 'Shade' in globals()
assert 'ZColor' in globals()

In [18]:
# Verborgene Überprüfungen 1

In [19]:
# Verborgene Überprüfungen 2

In [20]:
# Verborgene Überprüfungen 3

In [21]:
# Verborgene Überprüfungen 4

### Aufgabe 2b
Berechnen Sie aus der 3D-Punktwolke *pcl.laz* ein Digitales Oberflächenmodell (DOM) nach dem Verfahren von Hollaus et al. (2010). Verwenden Sie dazu das Modul [DSM](https://opals.geo.tuwien.ac.at/html/stable/ModuleDSM.html). Im Gegensatz zum DGM werden bei der Erstellung eines DOM alle Punkte des Datensatzes verwendet. Um ein zufriedenstellendes Ergebnis zu erhalten sollten folgende Parameter des Moduls eingestellt werden:
- gridSize
- neighbours
- searchRadius 
- maxSigma

Erstellen Sie analog zu *Aufgabe 2a* eine Schummerung und Höhenkodierung für das DOM. Verwenden Sie dazu Parameter Ihrer Wahl (die ein visuell ansprechendes Ergebnis erzeugen) und speichern Sie all Ihre Ergebnisse unter folgenden Dateinamen ab:
- Oberflächenmodell: *dom.tif*
- Schummerung: *dom_hillshade.tif*
- Höhenkodierung: *dom_zcolor.tif*

![](DOM_bsp.jpeg)

In [22]:
#
# Import Statements
#
import rasterio
import fiona
from os import path

In [23]:
# YOUR CODE HERE
dsm = DSM.DSM(inFile = 'pcl.odm', outFile = 'dom.tif')
dsm.gridSize = 0.4             #Gridsize etwas größer als bei DGM
dsm.neighbours = 16      #höhere Werte sind besser, aber nicht zu hoch
dsm.searchRadius = 5
dsm.maxSigma = 0.2       #je höher, desto unschärfer wird es
dsm.run()

Shade.Shade(inFile = 'dom.tif', outFile = 'dom_hillshade.tif', sunPosition = [350,50], shading = opals.Types.ShadingMethod.tanaka).run()

ZColor.ZColor(inFile = 'dom.tif', outFile = 'dom_zcolor.tif', palFile = 'standardPal.xml', interval = 2).run()

[Parameter(path=inFile, values=dom.tif, type=Path, source=interFace),
 Parameter(path=outFile, values=dom_zcolor.tif, type=Path, source=interFace),
 Parameter(path=oFormat, values=GTiff, type=String, source=estimated),
 Parameter(path=pixelSize, values=0.400000006, type=float, source=estimated),
 Parameter(path=interval, values=2, type=Vector<float>, source=interFace),
 Parameter(path=nClasses, values=, type=uint32, source=unknown),
 Parameter(path=palFile, values=standardPal.xml, type=Path, source=interFace),
 Parameter(path=legend, values=file=dom_zcolorLegend.svg, type=ZColorLegend, source=estimated),
 Parameter(path=scalePal, values=, type=ZColorScalePal, source=unknown),
 Parameter(path=offsetPal, values=0, type=float, source=estimated),
 Parameter(path=zRange, values=, type=Array<float,2>, source=unknown),
 Parameter(path=limit, values=, type=GridLimit, source=unknown),
 Parameter(path=resampling, values=, type=ResamplingMethod, source=unknown),
 Parameter(path=band, values=0, ty

In [24]:
# Sichtbare Überprüfungen
assert path.exists('dom.tif')
#
assert 'Import' in globals()
assert 'DSM' in globals()
assert 'Shade' in globals()
assert 'ZColor' in globals()

In [25]:
# Verborgene Überprüfungen 1

In [26]:
# Verborgene Überprüfungen 2

### Aufgabe 2c
Leiten Sie aus DGM und DOM ein normalisiertes Oberflächenmodell (nDOM) ab. Hierbei wird das DGM vom DOM subtrahiert um lediglich die Objekthöhen ohne Geländehöhe zu erhalten. Verwenden Sie dazu das Modul [Algebra](https://opals.geo.tuwien.ac.at/html/stable/ModuleAlgebra.html) und stellen Sie sicher, dass im nDOM keine negativen Höhen auftreten. Wählen Sie zur Berechnung des nDOM die gleiche Gitterweite wie jene des DGM. Visualisieren Sie das nDOM als höhenkodierte Rastergrafik mit einer Farbpalette ihrer Wahl (z.B. colorBrewer_SpectralPal.xml). Ermitteln Sie abschließend die mittlere Baumhöhe und das Vegetationsvolumen. Verwenden Sie dafür das Modul Histo und das zuvor erstellte nDOM.

Speichern Sie Ihre Ergebnisse unter folgenden Namen ab:
- Normalisiertes Oberfläcenmodell: *ndom.tif*
- Höhenkodierung: *ndom_zcolor.tif*

![](nDOM_bsp.jpeg)

In [27]:
# YOUR CODE HERE
alg = Algebra.Algebra(inFile = ['dgm.tif', 'dom.tif'], outFile = 'ndom.tif', gridSize = 0.25)
alg.formula = 'r[1]>=r[0] && r[0]>=0 ? r[1]-r[0] : 0'
alg.run()

ZColor.ZColor(inFile = 'ndom.tif', palFile = 'colorBrewer_SpectralPal.xml', scalePal = '0 30', interval = [0.25, 2]).run()

his = Histo.Histo(inFile = 'ndom.tif')
his.run()

mittlere_baumhoehe = his.histogram[0].getMean()
flaeche = his.histogram[0].getCountUsed() / mittlere_punktdichte
vegetationsvolumen = mittlere_baumhoehe * flaeche
print('Mittlere Baumhöhe: ', mittlere_baumhoehe, 'm','\nVegetationsvolumen: ', vegetationsvolumen, 'm³')

Mittlere Baumhöhe:  5.880184496405239 m 
Vegetationsvolumen:  55202.78526882162 m³


In [28]:
# Sichtbare Überprüfungen
assert path.exists('ndom.tif')
#
assert 'Algebra' in globals()
assert 'Histo' in globals()
assert 'ZColor' in globals()
#
assert 'mittel' in globals()
assert 'volumen' in globals()

AssertionError: 

In [ ]:
# Verborgene Überprüfungen 1

### Aufgabe 2d

Abschließend sollen im QGIS drei Karten visualisiert werden (Bezugssystem: ETRS 89, UTM 32N; EPSG: 25832). Die erste Karte soll Schummerung und Höhenkodierung des DGM enthalten. Legen Sie dazu die beiden TIFs übereinander und machen Sie das oberhalb liegende Bild transparent (50%). Die zweite Karte soll Schummerung und Höhenkodierung des DOM enthalten. Gehen sie beim Erstellen vor wie bei der ersten Karte. Die dritte Karte soll das Farbkodierte nDOM enthalten. Die Karten sollen folgende Elemente aufweisen:

- Daten (DGM, DOM und nDOM)
- Titel
- Legende, welche Sie durch das OPALS Modul ZColor erstellt haben (.svg-Datei)
- Nordpfeil
- Grid (nur Kartenrahmen mit Koordinatenzeichen)
- Maßstabsleiste (Größe sinnvoll wählen)


Speichern Sie Ihre Karten unter folgenden Namen ab:
- *DGM.pdf*
- *DOM.pdf*
- *nDOM.pdf*